In [1]:
import os
import numpy as np
import re
import string
import pandas as pd
import textblob
from sklearn import model_selection, preprocessing, linear_model
from sklearn import metrics
from sklearn import svm
from sklearn import naive_bayes
from sklearn import decomposition, ensemble

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import xgboost
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [2]:
#Get location
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'Costa Rican Household Poverty Level Prediction.ipynb',
 'Data Scientist Tutorial.ipynb',
 'Dataset',
 'Explanation of Embedding Layer.ipynb',
 'Home Credit Default Risk.ipynb',
 'House Prices Predict.ipynb',
 'Quora question Pairs.ipynb',
 'Recommendation System.ipynb',
 'SQL.ipynb',
 'Text Classification.ipynb',
 'Text data.ipynb',
 'Text Preprocessing.ipynb',
 'Token Text Data.ipynb',
 'Toxic Comment Classification Challenge.ipynb',
 'Twitter Sentiment Analysis.ipynb']

In [3]:
# Load Dataset
data = open('Dataset/corpus.txt', encoding='utf-8' ).read()


In [4]:
labels, texts = [], []
for i, line in enumerate(data.split('\n')):
    sentence = line.split()
    labels.append(sentence[0])
    texts.append(" ".join(sentence[1:]))
    
    
    
    

In [5]:
type(labels)

list

In [6]:
# Create a dataframe using texts labels
train = pd.DataFrame()
train['text'] = texts
train['label'] = labels
train.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [7]:
d = {'text': texts,
     'label': labels}
data1 = pd.DataFrame(d)
data1

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2
...,...,...
9995,A revelation of life in small town America in ...,__label__2
9996,Great biography of a very interesting journali...,__label__2
9997,Interesting Subject; Poor Presentation: You'd ...,__label__1
9998,Don't buy: The box looked used and it is obvio...,__label__1


In [8]:
train.head()

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [9]:
from sklearn.model_selection import train_test_split

# split and shuffle the dataset into trainig and validation dataset

train_x, valid_x, train_y, valid_y = train_test_split(train['text'],
                                                      train['label'], random_state=102)


In [10]:
train_x.shape, valid_x.shape

((7500,), (2500,))

In [11]:
train_y.unique()

array(['__label__1', '__label__2'], dtype=object)

In [12]:
# label encode the target variable
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit_transform(train['label'])

train_y = encoder.transform(train_y)
valid_y = encoder.transform(valid_y)

In [13]:
train_y

array([0, 1, 1, ..., 0, 1, 0])

### Feature Engineering

- Raw text data will be transformed into feature vectors


1. Bag of words - COunt vectors as features
2. TFIDF = Vectors as features
- Word level
- N-Gram level
- Character level
3. Word embedding as features
4. Text/NLP


#### dataset
- label
- text

In [14]:
# 1. Create a count vectorizer object

count = CountVectorizer(analyzer='word', token_pattern=r"\w{1,}")
count.fit(train['text'])


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [15]:
# transform teh training and validation data using count vectorizer object

train_count = count.transform(train_x)
valid_count = count.transform(valid_x)


In [26]:
count.get_feature_names()
train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
a = train_count.toarray()
a[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
train_count

<7500x31666 sparse matrix of type '<class 'numpy.int64'>'
	with 437119 stored elements in Compressed Sparse Row format>

In [16]:
# # 2. TF-IDF Vectors as features
# - word level
# - N-Gram level
# - Character level

# word level
tfidf = TfidfVectorizer(analyzer='word', token_pattern=r"\w{1,}", max_features=5000)
tfidf.fit(train['text'])

train_tfidf = tfidf.transform(train_x)
valid_tfidf = tfidf.transform(valid_x)


# n-gram level

tfidf_ngram = TfidfVectorizer(analyzer='word', token_pattern=r"w{1,}",
                              ngram_range=(2, 3), max_features=5000)
tfidf_ngram.fit(train['text'])

train_tfidf_ngram = tfidf_ngram.transform(train_x)
valid_tfidf_ngram = tfidf_ngram.transform(valid_x)


# character level

tfidf_chars = TfidfVectorizer(analyzer='char', token_pattern=r"\w{1,}",
                              ngram_range=(2, 3), max_features=5000)
tfidf_chars.fit(train['text'])

train_tfidf_ngram = tfidf_chars.transform(train_x)
valid_tfidf_ngram = tfidf_chars.transform(valid_x)


### Word Embeddings

- A Word Embeddings i a from of representing words and documents using a dense vectoer representation.
-  The position of a word within the vector space is learned from text and is based on the words that surround the word when it is used
- Word embeddings can be trained using the input corpus itself 
- Or can be generated using pre trained word embeddings such as Glove, FastText, Word2Vec 

### How to use pre- trained word embeddings in the model

1. Loading the pre trained word embeddings
2. Create a tokenizer object
3. Transforming text documents to sequence of tokens and pad them
4. Create a mapping of token and their respective embeddings


In [18]:
# # load the pre-trained word-embedding vectors 
# embeddings_index = {}
# for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
#     values = line.split()
#     embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# # create a tokenizer 
# token = text.Tokenizer()
# token.fit_on_texts(trainDF['text'])
# word_index = token.word_index

# # convert text to sequence of tokens and pad them to ensure equal length vectors 
# train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
# valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# # create token-embedding mapping
# embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

### Text based Features
1. Word Count of documents - total number of words in the documents
2. Character count of the documents
3. Average Word Density of the ducoment - average length of the words used in the documents
4. Punctuation COunt in the conplete essay
5. Upper case count in the complete essay
6. Title word Count in the complete essay
7. Frequency distribution of Part of Speech Tags
- NOuns, Verb, Adj, Adv, Pronoun


In [19]:
train

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2
...,...,...
9995,A revelation of life in small town America in ...,__label__2
9996,Great biography of a very interesting journali...,__label__2
9997,Interesting Subject; Poor Presentation: You'd ...,__label__1
9998,Don't buy: The box looked used and it is obvio...,__label__1


In [21]:
train['char_count'] = train['text'].apply(len)
train['word_count'] = train['text'].apply(lambda x: len(x.split()))
train['word_density'] = train['char_count'] / (train['word_count'] + 1)
train['punctuation_count'] = train['text'].apply(lambda x: len(''.join(_ for _ in x if _ in string.punctuation)))
train['title_word_count'] = train['text'].apply(lambda x: len([word for word in x.split() if word.istitle()]))


In [22]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

train['noun_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'noun'))
train['verb_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'verb'))
train['adj_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'adj'))
train['adv_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'adv'))
train['pron_count'] = train['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [23]:
train.head()

,text,label,char_count,word_char,word_count,word_density,punctuation_count,title_word_count,noun_count,verb_count,adj_count,adv_count,pron_count
0,Stuning even for the non-gamer: This sound tra...,__label__2,426,80,80,5.259259,11,10,20,15,6,6,11
1,The best soundtrack ever to anything.: I'm rea...,__label__2,509,97,97,5.193878,14,7,20,23,9,3,10
2,Amazing!: This soundtrack is my favorite music...,__label__2,760,129,129,5.846154,40,24,39,18,13,10,11
3,Excellent Soundtrack: I truly like this soundt...,__label__2,743,118,118,6.243697,33,52,52,12,9,2,7
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2,481,87,87,5.465909,22,30,31,13,7,2,9


In [24]:
train['text']

0       Stuning even for the non-gamer: This sound tra...
1       The best soundtrack ever to anything.: I'm rea...
2       Amazing!: This soundtrack is my favorite music...
3       Excellent Soundtrack: I truly like this soundt...
4       Remember, Pull Your Jaw Off The Floor After He...
                              ...                        
9995    A revelation of life in small town America in ...
9996    Great biography of a very interesting journali...
9997    Interesting Subject; Poor Presentation: You'd ...
9998    Don't buy: The box looked used and it is obvio...
9999    Beautiful Pen and Fast Delivery.: The pen was ...
Name: text, Length: 10000, dtype: object

### Topic Models as Features
- Topic modelling is a technique to identify the groups of words from a collection of documents that contains best information in the collection

-  LDA is an iterative model which starts from a fixed number of topics. Each topic is represented as a distribution over words, and each document is then represented as a distribution over topics. 

- 

In [32]:
# train a Latent Dirichlet Allocation
from sklearn.decomposition import LatentDirichletAllocation

lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online',
                                                   max_iter=20)
topics = lda_model.fit_transform(train_count)

topic_word = lda_model.components_
vocab = count.get_feature_names()

# view the topic models

n_top_words = 10

topic_summaries = []
for i, topic_dis in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dis)][:-(n_top_words+1): -1]
    topic_summaries.append(' '.join(topic_words))

In [33]:
topic_summaries

['voodoo scooter napoleon ulysses wished bela nbc dover differential writting',
 'female male henry italian successful rod gorgeous mrs interpretation ya',
 'rice thoroughly rabbit beads freud baseball cooker bore porn absurd',
 'the and a of to i this is it book',
 'the is of this a and cd s music album',
 'bands count build emarker blocks spending hurry chest rain juvenile',
 'cartridge theology sticks flint clippers wagner nb entitled recalled cowboy',
 'boots boot thus hiking damage waterproof printed police socks gluten',
 'african insights wars village bland buendia solitude recall rise devoid',
 'la de en y con el los sin que un',
 'relationship double grammar essential editor management snmp prehistoric enter merely',
 'countries fence germany systems map rigid tile criticize samsung cream',
 'recordings nirvana chase venus wit professor stewart mentions arab orchestra',
 'the to it is you and a for game this',
 'costume daily dishes pratchett waist cincher discworld pregnancy 

### Build Model 
#### Text Classification



In [34]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,
                is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # Predict the labels on the validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    return metrics.accuracy_score(predictions, valid_y)


In [37]:
# Naive Bayes
# on Count Vectors

naive_accuracy = train_model(naive_bayes.MultinomialNB(), train_count, train_y,
                             valid_count)
print(f" NB, Count Vectors: { naive_accuracy}")

# Naive Bayes on word level TFIDF vectors

accuracy = train_model(naive_bayes.MultinomialNB(), train_tfidf, train_y,
                       valid_tfidf)
print( f"NB, Wordlevel TF-IDF: {accuracy}")

 NB, Count Vectors: 0.8428
NB, Wordlevel TF-IDF: 0.8492


In [39]:
# Linear Classifier
accuracy = train_model(linear_model.LogisticRegression(), train_count,
                       train_y, valid_count)

print(f"LR, Count Vectors: {accuracy}")

# Linear Classifier on word level TF-IDF Vectors

accuracy = train_model(linear_model.LogisticRegression(),
                       train_tfidf, train_y, valid_tfidf)
print(f"LR, Wordlevel TF-IDF: { accuracy}")


LR, Count Vectors: 0.8592
LR, Wordlevel TF-IDF: 0.8696


In [40]:
# SVM Model

accuracy = train_model(svm.SVC(), train_tfidf, train_y, valid_tfidf)
print(f"SVM, WordLevel vectors: { accuracy}")

C:\Users\HT\Miniconda3\envs\cs_ftmle\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, WordLevel vectors: 0.4964


In [41]:
# Bagging model

accuracy = train_model(ensemble.RandomForestClassifier(),
                       train_count, train_y, valid_count)

print(f"RF, Count Vectors: { accuracy}")

# Random Forest on word level TF -IDF vectors
accuracy = train_model(ensemble.RandomForestClassifier(), train_tfidf,
                       train_y, valid_tfidf)
print(f"RF, WordLevel TF-IDF: { accuracy}")


C:\Users\HT\Miniconda3\envs\cs_ftmle\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, Count Vectors: 0.7364


C:\Users\HT\Miniconda3\envs\cs_ftmle\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, WordLevel TF-IDF: 0.7596


In [42]:
# Boosting Model
# Boosting mdoels are another type of ensemble models part of tree based models
# Boosting is a machine learning ensemble meta-algorithm for primarily reducing bias, 
#and also variance in supervised learning, and a family of machine learning algorithms 
#that convert weak learners to strong ones.



In [44]:
#Extreme Gradient Boosting on Count Vectors

accuracy = train_model(xgboost.XGBClassifier(), train_count.tocsc(),
                       train_y, valid_count.tocsc())
print(f"Xgb, Count Vectors: { accuracy}")

# XGB model on word Level TFIDF vectors

accuracy = train_model(xgboost.XGBClassifier(), train_tfidf.tocsc(),
                       train_y, valid_tfidf.tocsc())

print(f"XGB, WordLevel TF-IDF: {accuracy}")


Xgb, Count Vectors: 0.7968
XGB, WordLevel TF-IDF: 0.7976


In [52]:
# Shallow model
# Neural network contains mainly 3 typers of layers
# inputlayer, hiddenlayer, out layer

def create_model(input_size):
    #  create input layer
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    
    hidden_layer = layers.Dense(100, activation='relu')(input_layer)
    # create outlayer
    
    output_layer = layers.Dense(1, activation='sigmoid')(hidden_layer)
    
    classifier = models.Model(inputs=input_layer, outputs=output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss="binary_crossentropy")
    
    return classifier

In [48]:
train_tfidf.shape

(7500, 5000)

In [56]:
train_tfidf_ngram.shape

(7500, 5000)

In [57]:
train_tfidf_ngram

<7500x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3381727 stored elements in Compressed Sparse Row format>

In [63]:
# classifier = create_model(train_tfidf_ngram.shape[1])
# accuracy = train_model(classifier, train_tfidf_ngram, train_y, valid_tfidf_ngram,
#                        is_neural_net=True)

# print(f"NN, WordLevel TF-IDF Vectors: { accuracy}")

In [ ]:
# Deep neural network
# Convolutional Neural network


In [64]:
# create a tokenizer
token = text.Tokenizer()
token.fit_on_texts(train['text'])


In [66]:
# Convert text to sequence of tokens and pad them to ensure euqal length vectors

train_seq = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)
train_seq[0]

array([    3,    13,    29,   223,    75,     1,   317,     9,    33,
           3,    98,   747,    18,     1,    77,   243,     7,     9,
         173,     5,   163,   327,    75,     1,  3770,    23,    26,
        1055,    15,    21,     5,    21,   917,     6, 20901,     5,
          52,     7,   229,    55,   356,    61,    79,     8,    80,
          23,   181,     5,   412,    17,    54,   195,     4,  3487,
           5,   677,  5910,  1585,   719,  5831,    19,   164,     2,
           7,  2171,    10,   294,   102,     4,   641])

In [68]:
word_index = token.word_index
word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'a': 4,
 'to': 5,
 'of': 6,
 'it': 7,
 'this': 8,
 'is': 9,
 'in': 10,
 'for': 11,
 'that': 12,
 'was': 13,
 'book': 14,
 'you': 15,
 'not': 16,
 'but': 17,
 'with': 18,
 'on': 19,
 'my': 20,
 'have': 21,
 'as': 22,
 'are': 23,
 'one': 24,
 'be': 25,
 'so': 26,
 'all': 27,
 'if': 28,
 'very': 29,
 'like': 30,
 'read': 31,
 'good': 32,
 'great': 33,
 'at': 34,
 'movie': 35,
 'they': 36,
 'just': 37,
 'about': 38,
 'from': 39,
 'or': 40,
 'would': 41,
 'an': 42,
 'me': 43,
 'out': 44,
 'what': 45,
 'has': 46,
 'more': 47,
 'by': 48,
 'time': 49,
 'had': 50,
 'when': 51,
 'get': 52,
 'will': 53,
 "it's": 54,
 'up': 55,
 'there': 56,
 'no': 57,
 'only': 58,
 'your': 59,
 'can': 60,
 "don't": 61,
 'his': 62,
 'really': 63,
 'who': 64,
 'some': 65,
 'he': 66,
 'well': 67,
 'first': 68,
 'her': 69,
 'much': 70,
 'than': 71,
 'even': 72,
 'do': 73,
 'story': 74,
 'because': 75,
 'them': 76,
 'other': 77,
 'after': 78,
 'buy': 79,
 'we': 80,
 'were': 81,
 'too': 

In [70]:
# Convolutional Neural Network CNN
def create_cnn():
    # add an Input layer
    input_layer = layers.Input((70, ))
    
    #add the word embedding layer
    
    embedding_layer = layers.Embedding(len(word_index) + 1, 300,
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # add the convolutional layer
    
    conv_layer = layer.Convolution1D(100, 3, activation='relu')(embdding_layer)
    
    # add the pooling layers
    
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    
    #add the out layers
    output_layer1 = layers.Dense(50, activation='relu')(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(out_put_layer1)
    
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    
    # compile the model
    model = models.Model(inuts=input_layer, outputs=output_layer2)
    
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x,
                       is_neural_net=True)

print(f"CNN, Word embeddings: { accuracy}")
    
    
    

In [ ]:
#  RNN - LSTM
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

In [ ]:
# Recurrent Neural Network GRU
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-GRU, Word Embeddings",  accuracy)

In [72]:
# Bidirectional RNN
def create_bidirectional_rnn():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

    

In [ ]:
# Recurrent Convolutional Neural Network

def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accurac)